In [3]:
import os
import cv2
import numpy as np

# Paths
input_images_dir = 'C:\\Users\\Shreya\\Downloads\\Brain Tumer Segmentation'
input_masks_dir = 'C:\\Users\\Shreya\Downloads\\Brain Tumer Segmentation\\masks'
output_base_dir = 'C:\\Users\\Shreya\\Downloads\\Brain_tumour'

# Folders for splits
splits = {
    'train': (0, 2000),
    'val': (2000, 2500),
    'test': (2500, 3064)  # total = 2000+500+564
}

# Create output folders
for split in splits.keys():
    os.makedirs(os.path.join(output_base_dir, split, 'images'), exist_ok=True)
    os.makedirs(os.path.join(output_base_dir, split, 'masks'), exist_ok=True)

# Target size
target_size = (256, 256)

# Sort filenames to keep consistent order
filenames = sorted([f for f in os.listdir(input_images_dir) if f.endswith(('.png', '.jpg', '.jpeg'))])

# Process
for idx, filename in enumerate(filenames):
    # Load image and mask in GRAYSCALE
    img_path = os.path.join(input_images_dir, filename)
    mask_path = os.path.join(input_masks_dir, filename)
    
    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)

    # Resize
    img_resized = cv2.resize(img, target_size, interpolation=cv2.INTER_AREA)
    mask_resized = cv2.resize(mask, target_size, interpolation=cv2.INTER_NEAREST)

    # Normalize image
    img_normalized = img_resized.astype(np.float32) / 255.0

    # Denoise using fastNlMeansDenoising
    img_uint8 = (img_normalized * 255).astype(np.uint8)
    img_denoised = cv2.fastNlMeansDenoising(img_uint8, None, h=10, templateWindowSize=7, searchWindowSize=21)

    # Decide split
    if idx < splits['train'][1]:
        split = 'train'
    elif idx < splits['val'][1]:
        split = 'val'
    else:
        split = 'test'

    # Save
    img_save_path = os.path.join(output_base_dir, split, 'images', filename)
    mask_save_path = os.path.join(output_base_dir, split, 'masks', filename)

    cv2.imwrite(img_save_path, img_denoised)
    cv2.imwrite(mask_save_path, mask_resized)

print("✅ Preprocessing + Split complete! Images resized, normalized, denoised, and saved into train/val/test folders.")


ModuleNotFoundError: No module named 'cv2'